# Le compte est bon

[Le compte est bon](https://en.wikipedia.org/wiki/Des_chiffres_et_des_lettres#Le_compte_est_bon_&#40;%22the_total_is_right%22&#41;) is a TV game show where the goal is to get as close as possible to some target number by using 6 randomly drawn numbers and the 4 arithmetic operations.

For example if the target is __383__ and the numbers are __1, 3, 7, 7, 9, 25__, one possible solution (not necessarily the best) is: 

$$25 \times (7 + 9) - 7 \times (3 - 1) = 386$$

The purpose of this notebook is to write a program that solves this problem.

## Rules of the Game

* The target is a random number between $100$ and $999$.
* The 6 other numbers are drawn at random, without replacement from: $$\{1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100\}$$
* Each of the 6 numbers can only be used once, and not all numbers need to be used.
* Temporary and final results have to be positive integers.

## Draw

Python's random.sample function makes drawing a random game very easy:

In [1]:
import random

# For consistency.
random.seed(0)

def draw(n = 6):
    lo = 100
    hi = 999
    target = random.randint(lo, hi)
    cards = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100]
    return target, sorted(random.sample(cards, n))

For example:

In [2]:
target, cards = draw()
print('target: ', target, ', cards ', cards, sep='')

target: 964, cards [1, 5, 7, 7, 8, 9]


## Algorithm

We can solve this problem recursively as follows. If there is just 1 card, use that card. If we have 2 cards, try the 4 different operations. Otherwise, if we have $n > 1$ cards:
* choose 2 of the cards $a$ and $b$ and 1 of the operations $op$
* replace $a$ and $b$ by $a$ op $b$
* solve the problem on $n - 1$ cards

Note that $a+b$ and $a \times b$ are commutative and $a - b$ and $a \div b$ only make sense if $a \leq b$.

Therefore the number of operations is $N(n) \leq 4{n \choose 2} N(n-1)$, for $n > 2$:

\begin{align}
N(2) & \leq 4 \\
N(3) & \leq 48 \\
N(4) & \leq 1152 \\
N(5) & \leq 46{,}080 \\
N(6) & \leq 2{,}764{,}800
\end{align}

Though large, these numbers are not too large and we can do an extensive search. Let's first show how to find the best solution that uses __all__ the cards.

## Best solution with exactly n cards

In [3]:
# Returns the best solution using all cards.
def solve_using_all(target, cards):
    count = len(cards)
    assert count > 0
    if len(cards) == 1:        
        return cards[0], [cards[0]]
    best_reached = 0
    best_solution = []
    for i in range(count):
        for j in range(i + 1, count):
            for k in ['+', '-', '*', '/']:
                a = cards[i]
                b = cards[j]
                new_cards = list(cards)
                del new_cards[j], new_cards[i]
                if a < b: a, b = b, a
                if k == '+': result = a + b
                if k == '-': result = a - b
                if k == '*': result = a * b
                if k == '/':
                    if b > 0 and a % b == 0:
                        result = a // b
                    else:
                        continue
                new_cards.append(result)
                reached, solution = solve_using_all(target, new_cards)
                if abs(reached - target) < abs(best_reached - target):
                    best_reached = reached
                    best_solution = [[a, k, b, '=', result]]
                    best_solution.extend(solution)
                if reached == target:
                    return best_reached, best_solution
    return best_reached, best_solution

def print_solution(solution):
    for operation in solution:
        if type(operation) is int:
            if len(solution) == 1:
                print(operation)
        else:
            for token in operation:
                print(token, end = ' ')
        print()
        
target, cards = draw()
print(target, cards)
best_reached, best_solution = solve_using_all(target, cards)
print(best_solution)

514 [4, 5, 6, 8, 9, 10]
[[5, '+', 4, '=', 9], [9, '*', 6, '=', 54], [54, '+', 9, '=', 63], [63, '*', 8, '=', 504], [504, '+', 10, '=', 514], 514]


In [4]:
def test(target, cards, expect):
    reached, solution = solve_using_all(target, cards)
    assert(abs(target - reached) == abs(target - expect))

def unit_tests():
    test(1, [1], 1)
    test(1, [2], 2)
    test(1, [3], 3)
    test(10, [1, 1], 2)
    test(10, [1, 1, 1], 3)
    test(10, [1, 1, 1, 1], 4)
    test(10, [1, 1, 1, 1, 1], 6)
    test(10, [1, 1, 1, 1, 1, 1], 9)
    test(5, [1, 1, 1, 1, 1, 1], 5)
    test(1, [1, 1, 1, 1, 1, 1], 1)
    test(2, [1, 1, 1, 1, 1, 1], 2)

unit_tests()

## Shortest Solution

Now that we know how to compute the best solution that uses exactly $n$ cards, we can compute, among the best solutions, the shortest one:

In [5]:
# Returns the number of 1's in the binary representation of n.
def count_1_bits(n):
    count = 0
    while n > 0:
        if n & 1 != 0:
            count += 1
        n = n >> 1
    return count

# Returns a subset of a set, by using mask to select its elements.
def make_subset(set, mask):
    subset = []
    index = 0
    while mask > 0:
        if mask & 1 != 0:
            subset.append(set[index])
        index += 1
        mask = mask >> 1
    return subset

# Returns the best solution using as few cards as possible.
def solve_using_any(target, all_cards):
    best_reached = 0
    best_solution = []
    n = len(all_cards)
    for i in range(1, n + 1):
        for mask in range(2 ** n):
            if count_1_bits(mask) == i:
                cards = make_subset(all_cards, mask)
                reached, solution = solve_using_all(target, cards)
                if abs(reached - target) < abs(best_reached - target):
                    best_reached = reached
                    best_solution = solution
                if reached == target:
                    return best_reached, best_solution
    return best_reached, best_solution

# Alias.
def solve(target, cards):
    return solve_using_any(target, cards)
            
target, cards = 383, [1, 3, 7, 7, 9, 25]
print('target =', target)
print('cards =', cards)
print()
reached, solution = solve(target, cards)
print_solution(solution)

target = 383
cards = [1, 3, 7, 7, 9, 25]

9 + 3 = 12 
25 + 7 = 32 
32 * 12 = 384 
384 - 1 = 383 

